In [4]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.functions import lit, isnull, when, count, col, regexp_extract, concat_ws, to_date, expr, quarter, when, date_add, year, month, day, dayofweek, broadcast, avg, min, max, like
import pydeequ

# Define spark session config
spark_configs = {
    'spark.master': 'spark://spark-iceberg:7077',
    'spark.sql.catalog.prod': 'org.apache.iceberg.spark.SparkCatalog',
    'spark.sql.catalog.prod.io-impl': 'org.apache.iceberg.aws.s3.S3FileIO',
    'spark.sql.catalog.prod.s3.endpoint': 'http://minio:9000',
    'spark.sql.catalog.prod.type': 'rest',
    'spark.sql.catalog.prod.uri': 'http://rest:8181',
    'spark.sql.catalog.prod.warehouse': 's3://warehouse',
    'spark.sql.defaultCatalog': 'prod',
    'spark.driver.memory': '1G',
    'spark.executor.memory': '1G',
    
    "spark.jars.packages": pydeequ.deequ_maven_coord,
    "spark.jars.excludes": 'net.sourceforge.f2j:arpack_combined_all-0.1'
}

# Initialize SparkSession
spark = (
    SparkSession
    .builder
    .appName('Agg Fact Testing')
    .config(map=spark_configs)
    .getOrCreate()
)

In [5]:
def generate_dim_dates_df(spark: SparkSession) -> DataFrame:
    # Initialize dates_df
    dates_df = spark.range(365) \
        .withColumn('date', expr('date_add("2015-01-01", CAST(id AS INT))')) \
        .withColumn('year', year('date')) \
        .withColumn('month', month('date')) \
        .withColumn('day', day('date')) \
        .withColumn('day_of_week', dayofweek('date')) \
        .withColumn('quarter', quarter('date')) \
        .drop('id')

    # List of U.S. federal holidays
    us_holidays_2015 = [
        ("2015-01-01", "New Year's Day"),
        ("2015-01-19", "Martin Luther King Jr. Day"),
        ("2015-02-16", "Presidents' Day"),
        ("2015-05-25", "Memorial Day"),
        ("2015-07-04", "Independence Day"),
        ("2015-09-07", "Labor Day"),
        ("2015-10-12", "Columbus Day"),
        ("2015-11-11", "Veterans Day"),
        ("2015-11-26", "Thanksgiving Day"),
        ("2015-12-25", "Christmas Day"),
    ]
    
    # Create holidays_df and cast date from STRING to DATE type
    holidays_df = spark.createDataFrame(us_holidays_2015, ['holiday_date', 'holiday_name'])
    holidays_df = holidays_df.withColumn('holiday_date', to_date('holiday_date'))

    # Join holidays to date_df and add is_holiday column
    dates_df = dates_df \
        .join(
            broadcast(holidays_df),
            dates_df.date == holidays_df.holiday_date,
            'left'
        ) \
        .withColumn(
            'is_holiday',
            when(col('holiday_name').isNotNull(), lit(True)).otherwise(lit(False))
        ) \
        .drop('holiday_date') \
        .sort('date')

    # Rearrange date to be first column,
    dates_df = dates_df.select('date', *[col(c) for c in dates_df.columns if c != 'date'])
    return dates_df

dates_df = generate_dim_dates_df(spark)

In [6]:
# Read flights table
flights_df = spark.table('prod.db.fact_flights')

# Join dim_date table to flights
flights_df = flights_df \
    .join(
        broadcast(dates_df),
        ['date']
    )

flights_df.createOrReplaceTempView('flights')

flights_df.show(10)

25/01/06 04:14:57 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+----------+----+-----+---+-----------+-------+------------+----------+
|      date|airline|flight_number|tail_number|origin_airport|destination_airport|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|is_delayed|year|month|day|day_of_week|quarter|holiday_name|is_holiday|
+----------+-------+-------------+-----------+--------------+-------------------+-----------------

In [7]:
def get_aggregation_level(columns: list[str]):
    # TODO add comments
    # Returns col
    agg_level = concat_ws(
        '_',
        *[when(col(c).isNotNull(), lit(c)) for c in columns]
    )

    all_nulls = lit(True)
    for c in columns:
        all_nulls &= col(c).isNull()
    agg_level = when(all_nulls, 'all').otherwise(agg_level)
    return agg_level
    

In [8]:
agg_query = """
SELECT
    -- Grouping columns
    year,
    month,
    day_of_week,
    
    airline,
    origin_airport,

    -- Aggregation columns
    COUNT(*) AS total_flights,
    
    COUNT(CASE WHEN is_delayed = 1 THEN 1 END) AS delayed_flights,
    delayed_flights / total_flights AS delayed_rate,
    AVG(departure_delay) AS avg_delay_time,

    COUNT(CASE WHEN cancelled = 1 THEN 1 END) AS cancelled_flights,
    cancelled_flights / total_flights AS cancelled_rate,

    -- Total cancellations for each reason
    COUNT(CASE WHEN cancellation_reason = 'A' THEN 1 END) AS cancellations_A,
    COUNT(CASE WHEN cancellation_reason = 'B' THEN 1 END) AS cancellations_B,
    COUNT(CASE WHEN cancellation_reason = 'C' THEN 1 END) AS cancellations_C,
    COUNT(CASE WHEN cancellation_reason = 'D' THEN 1 END) AS cancellations_D,

    -- Percentage of cancellations by each reason
    cancellations_A / cancelled_flights AS percent_cancellations_A,
    cancellations_B / cancelled_flights AS percent_cancellations_B,
    cancellations_C / cancelled_flights AS percent_cancellations_C,
    cancellations_D / cancelled_flights AS percent_cancellations_D
    
FROM flights

-- Group by various levels of granularity using GROUPING SETS
GROUP BY GROUPING SETS (
    -- Group by year
    (year), 

    -- Group by year and month
    (year, month),

    -- Group by day of the week
    (day_of_week),
    
    -- Group by airline at different levels
    (airline), 
    (airline, year), 
    (airline, year, month),

    -- Group by origin airport at different levels
    (origin_airport), 
    (origin_airport, year), 
    (origin_airport, year, month)
)
"""

agg_df = spark.sql(agg_query)

# Define aggregation level columns for time and non-time columns
time_cols = ['year', 'month', 'day_of_week']
non_time_cols = ['airline', 'origin_airport']

# Get aggregation level for time and non-time columns
time_agg_level = get_aggregation_level(time_cols)
agg_level = get_aggregation_level(non_time_cols)

# Add aggregation level columns to the DataFrame
agg_df = agg_df \
    .withColumn('time_agg_level', time_agg_level) \
    .withColumn('agg_level', agg_level)


print(agg_df.count())
agg_df.select(col('agg_level')).distinct().sort('agg_level').show(truncate=False)
agg_df.sort('agg_level').show(50)

5188


+--------------+
|agg_level     |
+--------------+
|airline       |
|all           |
|origin_airport|
+--------------+



+----+-----+-----------+-------+--------------+-------------+---------------+-------------------+-------------------+-----------------+--------------------+---------------+---------------+---------------+---------------+-----------------------+-----------------------+-----------------------+-----------------------+--------------+---------+
|year|month|day_of_week|airline|origin_airport|total_flights|delayed_flights|       delayed_rate|     avg_delay_time|cancelled_flights|      cancelled_rate|cancellations_A|cancellations_B|cancellations_C|cancellations_D|percent_cancellations_A|percent_cancellations_B|percent_cancellations_C|percent_cancellations_D|time_agg_level|agg_level|
+----+-----+-----------+-------+--------------+-------------+---------------+-------------------+-------------------+-----------------+--------------------+---------------+---------------+---------------+---------------+-----------------------+-----------------------+-----------------------+-----------------------+

In [20]:
agg_df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- airline: string (nullable = true)
 |-- origin_airport: string (nullable = true)
 |-- total_flights: long (nullable = false)
 |-- delayed_flights: long (nullable = false)
 |-- delayed_rate: double (nullable = true)
 |-- avg_delay_time: double (nullable = true)
 |-- cancelled_flights: long (nullable = false)
 |-- cancelled_rate: double (nullable = true)
 |-- cancellations_A: long (nullable = false)
 |-- cancellations_B: long (nullable = false)
 |-- cancellations_C: long (nullable = false)
 |-- cancellations_D: long (nullable = false)
 |-- percent_cancellations_A: double (nullable = true)
 |-- percent_cancellations_B: double (nullable = true)
 |-- percent_cancellations_C: double (nullable = true)
 |-- percent_cancellations_D: double (nullable = true)
 |-- time_agg_level: string (nullable = false)
 |-- agg_level: string (nullable = false)



In [14]:
# agg_df.filter('agg_level NOT IN ("all", "airline", "origin_airport")').show()
agg_df.filter('agg_level = "0.9633770239013107"').show()

+----+-----+-----------+-------+--------------+-------------+---------------+------------+--------------+-----------------+--------------+---------------+---------------+---------------+---------------+-----------------------+-----------------------+-----------------------+-----------------------+--------------+---------+
|year|month|day_of_week|airline|origin_airport|total_flights|delayed_flights|delayed_rate|avg_delay_time|cancelled_flights|cancelled_rate|cancellations_A|cancellations_B|cancellations_C|cancellations_D|percent_cancellations_A|percent_cancellations_B|percent_cancellations_C|percent_cancellations_D|time_agg_level|agg_level|
+----+-----+-----------+-------+--------------+-------------+---------------+------------+--------------+-----------------+--------------+---------------+---------------+---------------+---------------+-----------------------+-----------------------+-----------------------+-----------------------+--------------+---------+
+----+-----+-----------+----

25/01/06 05:04:49 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/blockmgr-fd2bdf34-1f0e-4099-a5ee-1bd5fe226538. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/blockmgr-fd2bdf34-1f0e-4099-a5ee-1bd5fe226538
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:173)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:368)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1$adapted(DiskBlockManager.scala:364)
	at scala.collection.IndexedSeqOptimize

# Data Quality Profiler for agg_fact_flights
Possibly bug in Deequ preventing us from running profiler and constraint suggestion on agg_df (https://github.com/awslabs/deequ/issues/592)

In [ ]:
from pydeequ.profiles import *

# Profiling all the columns: ColumnProfilerRunner.onData returns a ColumnProfilerRunBuilder
result = ColumnProfilerRunner(spark) \
    .onData(agg_df) \
    .restrictToColumns(['time_agg_level']) \
    .run()

# printing all the columns and their corresponding profiled data.
for col_name, profile in result.profiles.items():
    print(profile)

In [ ]:
from pydeequ.suggestions import *

# Run constraint suggestion
suggestionResult = ConstraintSuggestionRunner(spark) \
    .onData(agg_df) \
    .addConstraintRule(CompleteIfCompleteRule()) \
    .run()


In [ ]:
# Print constraint suggestions
for constraint in suggestionResult['constraint_suggestions']:
    print(constraint["code_for_constraint"])
    print()

# Data Quality Checks

In [30]:
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "agg_fact_flights checks")

check.isEqual()


AttributeError: 'Check' object has no attribute 'isEqual'

# Business Analysis

## Total Flight Stats

In [57]:
agg_df.filter('agg_level = "year"').show()

+----+-----+-----------+-------+--------------+-------------+---------------+-------------------+-----------------+-----------------+--------------------+---------------+---------------+---------------+---------------+-----------------------+-----------------------+-----------------------+-----------------------+---------+
|year|month|day_of_week|airline|origin_airport|total_flights|delayed_flights|       delayed_rate|   avg_delay_time|cancelled_flights|      cancelled_rate|cancellations_A|cancellations_B|cancellations_C|cancellations_D|percent_cancellations_A|percent_cancellations_B|percent_cancellations_C|percent_cancellations_D|agg_level|
+----+-----+-----------+-------+--------------+-------------+---------------+-------------------+-----------------+-----------------+--------------------+---------------+---------------+---------------+---------------+-----------------------+-----------------------+-----------------------+-----------------------+---------+
|2015| NULL|       NULL| 

## How does the overall flight volume vary by month? By day of week?

In [60]:
agg_df.filter('agg_level = "year_month"').sort('month').select(['year', 'month', 'total_flights']).show()
agg_df.filter('agg_level = "day_of_week"').sort('day_of_week').select(['day_of_week', 'total_flights']).show()

+----+-----+-------------+
|year|month|total_flights|
+----+-----+-------------+
|2015|    1|        46810|
|2015|    2|        42769|
|2015|    3|        49962|
|2015|    4|        48252|
|2015|    5|        49767|
|2015|    6|        50291|
|2015|    7|        51807|
|2015|    8|        51416|
|2015|    9|        46707|
|2015|   10|        48806|
|2015|   11|        46727|
|2015|   12|        47911|
+----+-----+-------------+



+-----------+-------------+
|day_of_week|total_flights|
+-----------+-------------+
|          1|        82017|
|          2|        85904|
|          3|        84462|
|          4|        85822|
|          5|        86894|
|          6|        86099|
|          7|        70027|
+-----------+-------------+



## What percentage of flights experienced a departure delay in 2015? Among those flights, what was the average delay time, in minutes?

In [62]:
agg_df.filter('agg_level = "year"').select(['year', 'delayed_rate', 'avg_delay_time']).show()

+----+-------------------+-----------------+
|year|       delayed_rate|   avg_delay_time|
+----+-------------------+-----------------+
|2015|0.36581702438814573|9.396523327726925|
+----+-------------------+-----------------+



## How does the % of delayed flights vary throughout the year? What about for flights leaving from Boston (BOS) specifically?



In [64]:
agg_df.filter('agg_level = "year_month"').select(['year', 'month', 'delayed_rate', 'avg_delay_time']).sort(['year', 'month']).show()

+----+-----+-------------------+------------------+
|year|month|       delayed_rate|    avg_delay_time|
+----+-----+-------------------+------------------+
|2015|    1|0.37983336893826103|10.075384480567848|
|2015|    2|0.40365685426360215|11.687378926460852|
|2015|    3|0.38585324846883634| 9.496233367451381|
|2015|    4| 0.3439235679350079| 7.574346217363129|
|2015|    5| 0.3627303233066088| 9.689226771685512|
|2015|    6|0.42641824580938936|13.978660943858442|
|2015|    7|0.40168317022796146|11.400552808813798|
|2015|    8| 0.3771588610549245|10.002964562677922|
|2015|    9| 0.2879868113987197| 5.003096774193549|
|2015|   10|  0.297975658730484| 5.071072883657764|
|2015|   11| 0.3233034434053117|  6.84846913420194|
|2015|   12|0.39458579449395753|11.853406364775067|
+----+-----+-------------------+------------------+



## Which airlines seem to be most and least reliable, in terms of on-time departure?

In [65]:
agg_df.filter('agg_level = "airline"').select(['airline', 'delayed_rate', 'avg_delay_time']).sort('airline').show()

+-------+-------------------+-------------------+
|airline|       delayed_rate|     avg_delay_time|
+-------+-------------------+-------------------+
|     AA|0.34133483820351534|  9.000586215559835|
|     AS| 0.2522248243559719| 1.6896754468485418|
|     B6| 0.3794398137718705| 11.254301957342898|
|     DL|0.32171158706018227|  7.427644424026645|
|     EV| 0.2974937387253271|  8.991135963177388|
|     F9| 0.3815150176678445| 12.697622750499889|
|     HA| 0.2669011332551778|0.13438683557529058|
|     MQ| 0.3180010198878123| 10.027902822436584|
|     NK| 0.4437920177006212| 15.850615571354258|
|     OO| 0.2950320076273495|  7.859373919059149|
|     UA|0.49861929130056964| 14.341261775397726|
|     US|0.31385292206169074|   6.30062194127243|
|     VX| 0.3775694787041605|  8.873712196743105|
|     WN| 0.4490405862995406|  10.63296968721066|
+-------+-------------------+-------------------+



## How many flights were cancelled in 2015? What % of cancellations were due to weather? What % were due to the Airline/Carrier?

In [66]:
agg_df.filter('agg_level = "year"').select(['year', 'cancelled_flights', 'percent_cancellations_A', 'percent_cancellations_B', 'percent_cancellations_C', 'percent_cancellations_D']).sort('airline').show()


+----+-----------------+-----------------------+-----------------------+-----------------------+-----------------------+
|year|cancelled_flights|percent_cancellations_A|percent_cancellations_B|percent_cancellations_C|percent_cancellations_D|
+----+-----------------+-----------------------+-----------------------+-----------------------+-----------------------+
|2015|             9044|     0.2758735072976559|     0.5466607695709863|     0.1774657231313578|                    0.0|
+----+-----------------+-----------------------+-----------------------+-----------------------+-----------------------+

